<a href="https://colab.research.google.com/github/jiuyuanx/11663-LLM/blob/main/LLAMA2_Fine_Tuning_Lora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q  torch peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 accelerate

import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline
)
from peft import LoraConfig
from trl import SFTTrainer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 109.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 105.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 46.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.9 MB/s eta 0:00:00


In [ ]:
import datasets

In [ ]:
# Dataset
data_name = "jiuyuan/train_cypher"
training_data = load_dataset(data_name, split="train")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
training_data

Dataset({
    features: ['input_text', 'output_text'],
    num_rows: 153
})

In [ ]:
# Model and tokenizer names
base_model_name = "NousResearch/Llama-2-7b-chat-hf"
refined_model = "llama-2-7b-mlabonne-enhanced"

# Tokenizer
llama_tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
llama_tokenizer.pad_token = llama_tokenizer.eos_token
llama_tokenizer.padding_side = "right"  # Fix for fp16

# Quantization Config
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False
)

# Model
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=quant_config,
    device_map={"": 0}
)
base_model.config.use_cache = False
base_model.config.pretraining_tp = 1

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

In [ ]:
# #Don't use this, extremely slow
# query = """Given my Neo4j database schema:[<Record nodes=[<Node element_id='-8' labels=frozenset({'conference'}) properties={'name': 'conference', 'indexes': [], 'constraints': []}>, <Node element_id='-6' labels=frozenset({'paper'}) properties={'name': 'paper', 'indexes': [], 'constraints': []}>, <Node element_id='-9' labels=frozenset({'affiliation'}) properties={'name': 'affiliation', 'indexes': [], 'constraints': []}>, <Node element_id='-7' labels=frozenset({'author'}) properties={'name': 'author', 'indexes': [], 'constraints': []}>, <Node element_id='-10' labels=frozenset({'domain'}) properties={'name': 'domain', 'indexes': [], 'constraints': []}>] relationships=[<Relationship element_id='-7' nodes=(<Node element_id='-7' labels=frozenset({'author'}) properties={'name': 'author', 'indexes': [], 'constraints': []}>, <Node element_id='-6' labels=frozenset({'paper'}) properties={'name': 'paper', 'indexes': [], 'constraints': []}>) type='author_write_paper' properties={'name': 'author_write_paper'}>, <Relationship element_id='-8' nodes=(<Node element_id='-6' labels=frozenset({'paper'}) properties={'name': 'paper', 'indexes': [], 'constraints': []}>, <Node element_id='-6' labels=frozenset({'paper'}) properties={'name': 'paper', 'indexes': [], 'constraints': []}>) type='paper_cite_paper' properties={'name': 'paper_cite_paper'}>, <Relationship element_id='-10' nodes=(<Node element_id='-6' labels=frozenset({'paper'}) properties={'name': 'paper', 'indexes': [], 'constraints': []}>, <Node element_id='-8' labels=frozenset({'conference'}) properties={'name': 'conference', 'indexes': [], 'constraints': []}>) type='paper_in_venue' properties={'name': 'paper_in_venue'}>, <Relationship element_id='-6' nodes=(<Node element_id='-7' labels=frozenset({'author'}) properties={'name': 'author', 'indexes': [], 'constraints': []}>, <Node element_id='-9' labels=frozenset({'affiliation'}) properties={'name': 'affiliation', 'indexes': [], 'constraints': []}>) type='author_in_affiliation' properties={'name': 'author_in_affiliation'}>, <Relationship element_id='-9' nodes=(<Node element_id='-6' labels=frozenset({'paper'}) properties={'name': 'paper', 'indexes': [], 'constraints': []}>, <Node element_id='-10' labels=frozenset({'domain'}) properties={'name': 'domain', 'indexes': [], 'constraints': []}>) type='paper_in_domain' properties={'name': 'paper_in_domain'}>]>] . Generate cypher query for the question: What are the top 5 main research areas in the 'ACL' conference? Only give cypher query, without any other words."""
# text_gen = pipeline(task="text-generation", model=base_model,return_full_text=False, tokenizer=llama_tokenizer, max_length=1024)
# output = text_gen(f"<s>[INST] {query} [/INST]")
# print(output[0]['generated_text'])

In [ ]:
def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['input_text'])):
        text = f"### Question: {example['input_text'][i]}\n ### Answer: {example['output_text'][i]}</s>"
        output_texts.append(text)
    return output_texts


In [ ]:
import pandas as pd
df_test_all = pd.read_csv("llm_all_v2.csv")
df_test_all

,Unnamed: 0,prompt,palm_generations,question,cypher,test,fine_tuned_palm_generations,fine_tuned_llama_generations,fine_tuned_llama_cleaned_generations,llama_generations,llama_cleaned_generations,gpt_generations,gpt_fine_tuned_generations
0,0,Given my Neo4j database schema:[<Record nodes=...,MATCH (p:paper) RETURN count(p),How many papers are in the database?,MATCH (p:paper) RETURN count(p) AS num_papers,1,MATCH (p:paper)\nRETURN COUNT(p),<s> Generate cypher query for the question: Ho...,MATCH (p:paper) RETURN count(p) AS num_papers,"To answer the question ""How many papers are ...","\nMATCH (p:Paper) RETURN p.id, COUNT(p)\n",MATCH (p:paper)\nRETURN count(p),MATCH (p:paper) RETURN count(p) AS num_papers
1,1,Given my Neo4j database schema:[<Record nodes=...,MATCH (a:author) RETURN count(a),Count the number of authors.,MATCH (a:Author) RETURN count(a) AS authorCount,1,MATCH (a:author)\nRETURN COUNT(a) AS authorCount,<s> Generate cypher query for the question: Co...,MATCH (a:Author)\nRETURN count(a) AS authorCount,Sure! Here's a Cypher query to count the num...,\nMATCH (a:Author) RETURN COUNT(a)\n,MATCH (a:author)\nRETURN count(a) AS authorCount,MATCH (a:Author) RETURN count(a) AS authorCount
2,2,Given my Neo4j database schema:[<Record nodes=...,MATCH (c:conference) RETURN count(c),How many conferences are listed?,MATCH (c:conference) RETURN count(c) AS countC...,0,MATCH (c:conference)\nRETURN COUNT(c),<s> Generate cypher query for the question: Ho...,MATCH (c:conference) RETURN count(c) AS countC...,"Sure! To answer the question ""How many confere...",\nMATCH (c:Conference) RETURN c.name\n,MATCH (c:conference)\nRETURN COUNT(c) as confe...,MATCH (c:conference) RETURN count(c) AS countC...
3,3,Given my Neo4j database schema:[<Record nodes=...,"MATCH (a:author {name: ""Daphne Koller""})-[:aut...",Find all papers by a 'Daphne Koller',MATCH (a:author {name:'Daphne Koller'})-[:auth...,0,MATCH (a:author {name: 'Daphne Koller'})-[:au...,<s> Generate cypher query for the question: Fi...,MATCH (a:author {name:'Daphne Koller'})-[:auth...,"To find all papers authored by ""Daphne Koller""...","\nMATCH (a:Author {name: ""Daphne Koller""})-[:A...",MATCH (a:author {name: 'Daphne Koller'})-[:aut...,MATCH (a:author {name:'Daphne Koller'})-[:auth...
4,4,Given my Neo4j database schema:[<Record nodes=...,MATCH (p:paper)-[r:paper_cite_paper]->(p2:pape...,How many papers cite at least one other paper?,MATCH (p:paper)-[:paper_cite_paper]->(:paper)\...,1,MATCH (p:paper)-[:paper_cite_paper]->(:paper)...,<s> Generate cypher query for the question: Ho...,MATCH (p:paper)-[:paper_cite_paper]->(:paper)\...,"To answer the question ""How many papers cite a...",MATCH (p1:Paper)-[:CITES {num_cites: > 1}]- (p...,MATCH (p1:paper)-[:paper_cite_paper]->(p2:pape...,MATCH (p:paper)-[:paper_cite_paper]->(:paper)\...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
214,214,Given my Neo4j database schema:[<Record nodes=...,MATCH (p:paper)-[:paper_in_domain]->(d:domain ...,Which papers on Maximum Likelihood have been p...,MATCH (p:paper)-[:paper_in_domain]->(d:domain ...,0,MATCH (p:paper)-[:paper_in_venue]->(c:confere...,<s> Generate cypher query for the question: Wh...,MATCH (p:paper)-[:paper_in_venue]->(:conferenc...,To find the papers on Maximum Likelihood tha...,\nMATCH (p:Paper)-[:PRESENTED_AT]->(conf:Confe...,MATCH (p:paper)-[:paper_in_venue]->(c:conferen...,MATCH (p:paper)-[:paper_in_venue]->(c:conferen...
215,215,Given my Neo4j database schema:[<Record nodes=...,MATCH (p:paper)-[:paper_in_domain]->(d:domain)...,Identify recent popular papers in Augmented Re...,MATCH (p:paper)-[:paper_in_domain]->(d:domain ...,0,MATCH (p:paper)-[:paper_in_venue]->(c:conferen...,<s> Generate cypher query for the question: Id...,[],To identify recent popular papers in Augmented...,MATCH (paper:AugmentedReality)-[:PUBLISHED]->(...,MATCH (p:paper)-[:paper_in_venue]->(:conferenc...,MATCH (p:paper)-[:paper_in_domain]->(d:domain ...
216,216,Given my Neo4j database schema:[<Record nod

## **Fine tune**

In [ ]:
# LoRA Config
peft_parameters = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=8,
    bias="none",
    task_type="CAUSAL_LM"
)

# Training Params
train_params = TrainingArguments(
    output_dir="./results_modified",
    num_train_epochs=50,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

# Trainer
fine_tuning = SFTTrainer(
    model=base_model,
    train_dataset=training_data,
    formatting_func=formatting_prompts_func, #hereeee
    peft_config=peft_parameters,
    # dataset_text_field="text",
    tokenizer=llama_tokenizer,
    args=train_params
)

# Training
fine_tuning.train()

# Save Model
fine_tuning.model.save_pretrained(refined_model)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/153 [00:00<?, ? examples/s]

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
25,1.724500
50,0.660200
75,0.435500
100,0.342700
125,0.299900
150,0.279400
175,0.232200
200,0.204400
225,0.170500
250,0.153000


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.

In [ ]:
# fine_tuning.model.save_pretrained(refined_model)


In [ ]:
# # tokenizer = AutoTokenizer.from_pretrained(checkpoint)
# prompt = "Generate cypher query for the question: What are the top 5 main research areas in the 'ACL' conference? Only give cypher query, without any other words."
# # Generate predictions
# inputs = llama_tokenizer(prompt, return_tensors='pt')
# inputs = inputs.to("cuda")
# output = fine_tuning.model.generate(**inputs, max_new_tokens=100)
# response = llama_tokenizer.decode(output[0].tolist())

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


In [ ]:
# response

"<s> Generate cypher query for the question: What are the top 5 main research areas in the 'ACL' conference? Only give cypher query, without any other words.\nOnly give cypher query, without any other words.\nMATCH (p:paper)-[:paper_in_venue]->(:conference {name: 'ACL'}),\n      (p)-[:paper_in_domain]->(d:domain)\nRETURN d.name, COUNT(p) AS numPapers\nORDER BY numPapers DESC\nLIMIT 5\nRETURN d.name, numPapers\nAS top5MainRes"

## **Fine tuned Generations**

In [ ]:
from tqdm.notebook import tqdm
generations =[]
for i in tqdm(range(len(df_test_all)), "generating..."):
  prompt = "Generate cypher query for the question: "+df_test_all.iloc[i]['question']+" Only give cypher query, without any other words."
  # Generate predictions
  inputs = llama_tokenizer(prompt, return_tensors='pt')
  inputs = inputs.to("cuda")
  output = fine_tuning.model.generate(**inputs, max_new_tokens=150,temperature=0.2)
  response = llama_tokenizer.decode(output[0].tolist())
  # print(response)
  # break
  generations.append(response)
generations

generating...:   0%|          | 0/219 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will

["<s> Generate cypher query for the question: How many papers are in the database? Only give cypher query, without any other words.\nExplanation: The number of papers in the database can be calculated by counting the number of papers in the 'paper' node.\nExample: MATCH (p:paper) RETURN count(p) AS num_papers</s></s>",
 '<s> Generate cypher query for the question: Count the number of authors. Only give cypher query, without any other words.\n```\nMATCH (a:Author)\nRETURN count(a) AS authorCount</s>\n</query>\n```</s>',
 '<s> Generate cypher query for the question: How many conferences are listed? Only give cypher query, without any other words.\n> MATCH (c:conference) RETURN count(c) AS countConference</s></s>',
 "<s> Generate cypher query for the question: Find all papers by a 'Daphne Koller' Only give cypher query, without any other words.\n</a>\n```\nMATCH (a:author {name:'Daphne Koller'})-[:author_write_paper]->(p:paper)\nRETURN p.name</s>\n</s>\n</query>\n</answer>\n</question>\n<

In [ ]:
df_test_all['fine_tuned_llama_generations']=generations
df_test_all.to_csv('PALM2_LLAMA2_results.csv')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df_test_all

,Unnamed: 0,prompt,generations,question,cypher,test,fine_tuned_generations,fine_tuned_llama_generations
0,0,Given my Neo4j database schema:[<Record nodes=...,MATCH (p:paper) RETURN count(p),How many papers are in the database?,MATCH (p:paper) RETURN count(p) AS num_papers,1,MATCH (p:paper)\nRETURN COUNT(p),<s> Generate cypher query for the question: Ho...
1,1,Given my Neo4j database schema:[<Record nodes=...,MATCH (a:author) RETURN count(a),Count the number of authors.,MATCH (a:Author) RETURN count(a) AS authorCount,1,MATCH (a:author)\nRETURN COUNT(a),<s> Generate cypher query for the question: Co...
2,2,Given my Neo4j database schema:[<Record nodes=...,MATCH (c:conference) RETURN count(c),How many conferences are listed?,MATCH (c:conference) RETURN count(c) AS countC...,0,MATCH (c:conference)\nRETURN COUNT(c),<s> Generate cypher query for the question: Ho...
3,3,Given my Neo4j database schema:[<Record nodes=...,"MATCH (a:author {name: ""Daphne Koller""})-[:aut...",Find all papers by a 'Daphne Koller',MATCH (a:author {name:'Daphne Koller'})-[:auth...,0,MATCH (a:author {name: 'Daphne Koller'})-[:au...,<s> Generate cypher query for the question: Fi...
4,4,Given my Neo4j database schema:[<Record nodes=...,MATCH (p:paper)-[r:paper_cite_paper]->(p2:pape...,How many papers cite at least one other paper?,MATCH (p:paper)-[:paper_cite_paper]->(:paper)\...,1,MATCH (p:paper)-[:paper_cite_paper]->(p2:pape...,<s> Generate cypher query for the question: Ho...
...,...,...,...,...,...,...,...,...
214,214,Given my Neo4j database schema:[<Record nodes=...,MATCH (p:paper)-[:paper_in_domain]->(d:domain ...,Which papers on Maximum Likelihood have been p...,MATCH (p:paper)-[:paper_in_domain]->(d:domain ...,0,MATCH (p:paper)-[:paper_in_domain]->(d:domain...,<s> Generate cypher query for the question: Wh...
215,215,Given my Neo4j database schema:[<Record nodes=...,MATCH (p:paper)-[:paper_in_domain]->(d:domain)...,Identify recent popular papers in Augmented Re...,MATCH (p:paper)-[:paper_in_domain]->(d:domain ...,0,MATCH (p:paper)-[:paper_in_venue]->(c:confere...,<s> Generate cypher query for the question: Id...
216,216,Given my Neo4j database schema:[<Record nodes=...,MATCH (p:paper)-[:paper_in_domain]->(d:domain)...,What are the trending research topics in Super...,MATCH (p:paper)-[:paper_in_domain]->(d:domain ...,0,MATCH (d:domain {name: 'Supervised Learning'}...,<s> Generate cypher query for the question: Wh...
217,217,Given my Neo4j database schema:[<Record nodes=...,```\nMATCH (p:paper)-[:paper_in_domain]->(d:do...,List the latest papers in Bioinformatics with ...,MATCH (p:paper)-[:paper_in_domain]->(d:domain ...,0,MATCH (p:paper)-[:paper_in_domain]->(d:domain...,<s> Generate cypher query for the question: Li...


In [ ]:
df_test_all.iloc[10]['fine_tuned_llama_generations']

"<s> Generate cypher query for the question: What papers are Computer Science? Only give cypher query, without any other words.\n</a>\nMATCH (p:paper)-[:paper_in_domain]->(d:domain {name: 'Computer Science'})\nRETURN p.name</s></s>"

In [ ]:
pattern = r'MATCH.*?</s>'
df_test_all['fine_tuned_llama_cleaned_generations']= df_test_all['fine_tuned_llama_generations'].apply(
    lambda x:re.findall(pattern, x, re.DOTALL) )

In [ ]:
import re

pattern = r'MATCH.*?</s>'
text = """<s> Generate cypher query for the question: What papers are Computer Science? Only give cypher query, without any other words.
</a>
MATCH (p:paper)-[:paper_in_domain]->(d:domain {name: 'Computer Science'})
RETURN p.name</s></s>"""

matches = re.findall(pattern, text, re.DOTALL)[0].replace('</s>', '')

In [ ]:
matches

"MATCH (p:paper)-[:paper_in_domain]->(d:domain {name: 'Computer Science'})\nRETURN p.name</s>"

In [ ]:
df_test_all['fine_tuned_llama_cleaned_generations']

0      [MATCH (p:paper) RETURN count(p) AS num_papers...
1      [MATCH (a:Author)\nRETURN count(a) AS authorCo...
2      [MATCH (c:conference) RETURN count(c) AS count...
3      [MATCH (a:author {name:'Daphne Koller'})-[:aut...
4      [MATCH (p:paper)-[:paper_cite_paper]->(:paper)...
                             ...                        
214    [MATCH (p:paper)-[:paper_in_venue]->(:conferen...
215                                                   []
216    [MATCH (p:paper)-[:paper_in_domain]->(d:domain...
217    [MATCH (p:paper)-[:paper_in_domain]->(d:domain...
218                                                   []
Name: fine_tuned_llama_cleaned_generations, Length: 219, dtype: object

In [ ]:
df_test_all['fine_tuned_llama_cleaned_generations'] = df_test_all['fine_tuned_llama_cleaned_generations'].apply(
    lambda x: x[0].replace("</s>", '') if len(x)>0 else x)

In [ ]:
df_test_all['fine_tuned_llama_cleaned_generations']

0          MATCH (p:paper) RETURN count(p) AS num_papers
1       MATCH (a:Author)\nRETURN count(a) AS authorCount
2      MATCH (c:conference) RETURN count(c) AS countC...
3      MATCH (a:author {name:'Daphne Koller'})-[:auth...
4      MATCH (p:paper)-[:paper_cite_paper]->(:paper)\...
                             ...                        
214    MATCH (p:paper)-[:paper_in_venue]->(:conferenc...
215                                                   []
216    MATCH (p:paper)-[:paper_in_domain]->(d:domain ...
217    MATCH (p:paper)-[:paper_in_domain]->(d:domain ...
218                                                   []
Name: fine_tuned_llama_cleaned_generations, Length: 219, dtype: object

In [ ]:
df_test_all.to_csv('PALM2_LLAMA2_results.csv')

## **Generations Original model**

In [ ]:
schema="""[<Record nodes=[<Node element_id='-8' labels=frozenset({'conference'}) properties={'name': 'conference', 'indexes': [], 'constraints': []}>, <Node element_id='-6' labels=frozenset({'paper'}) properties={'name': 'paper', 'indexes': [], 'constraints': []}>, <Node element_id='-9' labels=frozenset({'affiliation'}) properties={'name': 'affiliation', 'indexes': [], 'constraints': []}>, <Node element_id='-7' labels=frozenset({'author'}) properties={'name': 'author', 'indexes': [], 'constraints': []}>, <Node element_id='-10' labels=frozenset({'domain'}) properties={'name': 'domain', 'indexes': [], 'constraints': []}>] relationships=[<Relationship element_id='-7' nodes=(<Node element_id='-7' labels=frozenset({'author'}) properties={'name': 'author', 'indexes': [], 'constraints': []}>, <Node element_id='-6' labels=frozenset({'paper'}) properties={'name': 'paper', 'indexes': [], 'constraints': []}>) type='author_write_paper' properties={'name': 'author_write_paper'}>, <Relationship element_id='-8' nodes=(<Node element_id='-6' labels=frozenset({'paper'}) properties={'name': 'paper', 'indexes': [], 'constraints': []}>, <Node element_id='-6' labels=frozenset({'paper'}) properties={'name': 'paper', 'indexes': [], 'constraints': []}>) type='paper_cite_paper' properties={'name': 'paper_cite_paper'}>, <Relationship element_id='-10' nodes=(<Node element_id='-6' labels=frozenset({'paper'}) properties={'name': 'paper', 'indexes': [], 'constraints': []}>, <Node element_id='-8' labels=frozenset({'conference'}) properties={'name': 'conference', 'indexes': [], 'constraints': []}>) type='paper_in_venue' properties={'name': 'paper_in_venue'}>, <Relationship element_id='-6' nodes=(<Node element_id='-7' labels=frozenset({'author'}) properties={'name': 'author', 'indexes': [], 'constraints': []}>, <Node element_id='-9' labels=frozenset({'affiliation'}) properties={'name': 'affiliation', 'indexes': [], 'constraints': []}>) type='author_in_affiliation' properties={'name': 'author_in_affiliation'}>, <Relationship element_id='-9' nodes=(<Node element_id='-6' labels=frozenset({'paper'}) properties={'name': 'paper', 'indexes': [], 'constraints': []}>, <Node element_id='-10' labels=frozenset({'domain'}) properties={'name': 'domain', 'indexes': [], 'constraints': []}>) type='paper_in_domain' properties={'name': 'paper_in_domain'}>]>]"""

In [ ]:
from tqdm.notebook import tqdm
generations_original =[]
for i in tqdm(range(len(df_test_all)), "generating..."):
  prompt = "Given the neo4j schema: "+schema+"\nGenerate cypher query for the question: "+df_test_all.iloc[i]['question']+" Only give cypher query, without any other words. Put </s> at the end of answer."
  # Generate predictions
  inputs = llama_tokenizer(prompt, return_tensors='pt')
  inputs = inputs.to("cuda")
  output = base_model.generate(**inputs, max_new_tokens=150,temperature=0.2)
  response = llama_tokenizer.decode(output[0].tolist())
  # print(response)
  # break
  generations_original.append(response)
generations_original

generating...:   0%|          | 0/219 [00:00<?, ?it/s]

["<s> Given the neo4j schema: [<Record nodes=[<Node element_id='-8' labels=frozenset({'conference'}) properties={'name': 'conference', 'indexes': [], 'constraints': []}>, <Node element_id='-6' labels=frozenset({'paper'}) properties={'name': 'paper', 'indexes': [], 'constraints': []}>, <Node element_id='-9' labels=frozenset({'affiliation'}) properties={'name': 'affiliation', 'indexes': [], 'constraints': []}>, <Node element_id='-7' labels=frozenset({'author'}) properties={'name': 'author', 'indexes': [], 'constraints': []}>, <Node element_id='-10' labels=frozenset({'domain'}) properties={'name': 'domain', 'indexes': [], 'constraints': []}>] relationships=[<Relationship element_id='-7' nodes=(<Node element_id='-7' labels=frozenset({'author'}) properties={'name': 'author', 'indexes': [], 'constraints': []}>, <Node element_id='-6' labels=frozenset({'paper'}) properties={'name': 'paper', 'indexes': [], 'constraints': []}>) type='author_write_paper' properties={'name': 'author_write_paper'}>

In [ ]:
df_test_all['llama_generations'] = generations_original

In [ ]:
df_test_all['llama_generations'][0]

"<s> Given the neo4j schema: [<Record nodes=[<Node element_id='-8' labels=frozenset({'conference'}) properties={'name': 'conference', 'indexes': [], 'constraints': []}>, <Node element_id='-6' labels=frozenset({'paper'}) properties={'name': 'paper', 'indexes': [], 'constraints': []}>, <Node element_id='-9' labels=frozenset({'affiliation'}) properties={'name': 'affiliation', 'indexes': [], 'constraints': []}>, <Node element_id='-7' labels=frozenset({'author'}) properties={'name': 'author', 'indexes': [], 'constraints': []}>, <Node element_id='-10' labels=frozenset({'domain'}) properties={'name': 'domain', 'indexes': [], 'constraints': []}>] relationships=[<Relationship element_id='-7' nodes=(<Node element_id='-7' labels=frozenset({'author'}) properties={'name': 'author', 'indexes': [], 'constraints': []}>, <Node element_id='-6' labels=frozenset({'paper'}) properties={'name': 'paper', 'indexes': [], 'constraints': []}>) type='author_write_paper' properties={'name': 'author_write_paper'}>,

In [ ]:
df_test_all.to_csv("modified.csv")

In [ ]:
pattern = r'MATCH.*?</s>'
df_test_all['llama_cleaned_generations']= df_test_all['llama_generations'].apply(
    lambda x:re.findall(pattern, x, re.DOTALL) )

df_test_all['llama_cleaned_generations'] = df_test_all['llama_cleaned_generations'].apply(
    lambda x: x[0].replace("</s>", '') if len(x)>0 else x)


In [ ]:
df_test_all.to_csv("PAML2_LLAMA2_results1.csv")
df_test_all.to_csv("/content/drive/MyDrive/cypher/PAML2_LLAMA2_results.csv")

In [ ]:
output = base_model.generate(**inputs, max_new_tokens=100,temperature=0.2)

In [ ]:
response = llama_tokenizer.decode(output[0].tolist())
response

'<s> Generate cypher query for the question: What is the total number of affiliations? Only give cypher query, without any other words. nobody has more than 5 affiliations.\n\nHere is the cypher query:\n```\nMATCH (p:Person)-[:AFFILIATION]->()\nRETURN p.affiliations.length\n```\nExplanation:\n\n* `MATCH` is used to find all the people in the graph.\n* `-[:AFFILIATION]->()` is a path query that finds all the affiliations of each person.\n* `RET'

In [ ]:
# #Don't use this, extremely slow
query = """Given my Neo4j database schema:[<Record nodes=[<Node element_id='-8' labels=frozenset({'conference'}) properties={'name': 'conference', 'indexes': [], 'constraints': []}>, <Node element_id='-6' labels=frozenset({'paper'}) properties={'name': 'paper', 'indexes': [], 'constraints': []}>, <Node element_id='-9' labels=frozenset({'affiliation'}) properties={'name': 'affiliation', 'indexes': [], 'constraints': []}>, <Node element_id='-7' labels=frozenset({'author'}) properties={'name': 'author', 'indexes': [], 'constraints': []}>, <Node element_id='-10' labels=frozenset({'domain'}) properties={'name': 'domain', 'indexes': [], 'constraints': []}>] relationships=[<Relationship element_id='-7' nodes=(<Node element_id='-7' labels=frozenset({'author'}) properties={'name': 'author', 'indexes': [], 'constraints': []}>, <Node element_id='-6' labels=frozenset({'paper'}) properties={'name': 'paper', 'indexes': [], 'constraints': []}>) type='author_write_paper' properties={'name': 'author_write_paper'}>, <Relationship element_id='-8' nodes=(<Node element_id='-6' labels=frozenset({'paper'}) properties={'name': 'paper', 'indexes': [], 'constraints': []}>, <Node element_id='-6' labels=frozenset({'paper'}) properties={'name': 'paper', 'indexes': [], 'constraints': []}>) type='paper_cite_paper' properties={'name': 'paper_cite_paper'}>, <Relationship element_id='-10' nodes=(<Node element_id='-6' labels=frozenset({'paper'}) properties={'name': 'paper', 'indexes': [], 'constraints': []}>, <Node element_id='-8' labels=frozenset({'conference'}) properties={'name': 'conference', 'indexes': [], 'constraints': []}>) type='paper_in_venue' properties={'name': 'paper_in_venue'}>, <Relationship element_id='-6' nodes=(<Node element_id='-7' labels=frozenset({'author'}) properties={'name': 'author', 'indexes': [], 'constraints': []}>, <Node element_id='-9' labels=frozenset({'affiliation'}) properties={'name': 'affiliation', 'indexes': [], 'constraints': []}>) type='author_in_affiliation' properties={'name': 'author_in_affiliation'}>, <Relationship element_id='-9' nodes=(<Node element_id='-6' labels=frozenset({'paper'}) properties={'name': 'paper', 'indexes': [], 'constraints': []}>, <Node element_id='-10' labels=frozenset({'domain'}) properties={'name': 'domain', 'indexes': [], 'constraints': []}>) type='paper_in_domain' properties={'name': 'paper_in_domain'}>]>] . Generate cypher query for the question: What are the top 5 main research areas in the 'ACL' conference? Only give cypher query, without any other words."""
text_gen = pipeline(task="text-generation", model=base_model,return_full_text=False, tokenizer=llama_tokenizer, max_length=1024)
o = text_gen(f"<s>[INST] {query} [/INST]")
print(o[0]['generated_text'])

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


  Here is a Cypher query that answers the question "What are the top 5 main research areas in the 'ACL' conference?":

MATCH (n:ACL)-[:CONFERENCE]->()
RETURN n.labels, n.properties.name
ORDER BY n.properties.name DESC
LIMIT 5

Explanation:

* The `MATCH` clause matches all nodes that have the label "ACL" and are connected to the root node of the graph by a "CONFERENCE" relationship.
* The `RETURN` clause returns the labels and properties of the nodes, specifically the "name" property.
* The `ORDER BY` clause sorts the results by the "name" property in descending order (i.


In [ ]:
schema="""[<Record nodes=[<Node element_id='-8' labels=frozenset({'conference'}) properties={'name': 'conference', 'indexes': [], 'constraints': []}>, <Node element_id='-6' labels=frozenset({'paper'}) properties={'name': 'paper', 'indexes': [], 'constraints': []}>, <Node element_id='-9' labels=frozenset({'affiliation'}) properties={'name': 'affiliation', 'indexes': [], 'constraints': []}>, <Node element_id='-7' labels=frozenset({'author'}) properties={'name': 'author', 'indexes': [], 'constraints': []}>, <Node element_id='-10' labels=frozenset({'domain'}) properties={'name': 'domain', 'indexes': [], 'constraints': []}>] relationships=[<Relationship element_id='-7' nodes=(<Node element_id='-7' labels=frozenset({'author'}) properties={'name': 'author', 'indexes': [], 'constraints': []}>, <Node element_id='-6' labels=frozenset({'paper'}) properties={'name': 'paper', 'indexes': [], 'constraints': []}>) type='author_write_paper' properties={'name': 'author_write_paper'}>, <Relationship element_id='-8' nodes=(<Node element_id='-6' labels=frozenset({'paper'}) properties={'name': 'paper', 'indexes': [], 'constraints': []}>, <Node element_id='-6' labels=frozenset({'paper'}) properties={'name': 'paper', 'indexes': [], 'constraints': []}>) type='paper_cite_paper' properties={'name': 'paper_cite_paper'}>, <Relationship element_id='-10' nodes=(<Node element_id='-6' labels=frozenset({'paper'}) properties={'name': 'paper', 'indexes': [], 'constraints': []}>, <Node element_id='-8' labels=frozenset({'conference'}) properties={'name': 'conference', 'indexes': [], 'constraints': []}>) type='paper_in_venue' properties={'name': 'paper_in_venue'}>, <Relationship element_id='-6' nodes=(<Node element_id='-7' labels=frozenset({'author'}) properties={'name': 'author', 'indexes': [], 'constraints': []}>, <Node element_id='-9' labels=frozenset({'affiliation'}) properties={'name': 'affiliation', 'indexes': [], 'constraints': []}>) type='author_in_affiliation' properties={'name': 'author_in_affiliation'}>, <Relationship element_id='-9' nodes=(<Node element_id='-6' labels=frozenset({'paper'}) properties={'name': 'paper', 'indexes': [], 'constraints': []}>, <Node element_id='-10' labels=frozenset({'domain'}) properties={'name': 'domain', 'indexes': [], 'constraints': []}>) type='paper_in_domain' properties={'name': 'paper_in_domain'}>]>]"""

In [ ]:
from tqdm.notebook import tqdm
generations_raw =[]
for i in tqdm(range(len(df_test_all)), "generating..."):
  query = "Given the neo4j schema: "+schema+"\nGenerate cypher query for the question: "+df_test_all.iloc[i]['question']+"\nOnly give cypher query. Put </s> at the end of answer."
  # Generate predictions
  # inputs = llama_tokenizer(prompt, return_tensors='pt')
  # inputs = inputs.to("cuda")
  # output = base_model.generate(**inputs, max_new_tokens=150,temperature=0.2)
  # response = llama_tokenizer.decode(output[0].tolist())
  text_gen = pipeline(task="text-generation", model=base_model,return_full_text=False, tokenizer=llama_tokenizer, max_length=1024, temperature=0.2)
  out = text_gen(f"<s>[INST] {query} [/INST]")
  response = out[0]['generated_text']
  # print(response)
  # break
  generations_raw.append(response)
generations_raw

generating...:   0%|          | 0/219 [00:00<?, ?it/s]

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


['  To answer the question "How many papers are in the database?", you can use a Cypher query like this:\n```\nMATCH (p:Paper) RETURN p.id, COUNT(p)\n```\nThis query matches all papers in the database and returns their IDs and the count of papers. The `COUNT(p)` function returns the number of papers in the database.\n\nHere\'s a breakdown of the query:\n\n* `MATCH (p:Paper)`: This line matches all papers in the database. The `p` variable is assigned the papers that match the pattern.\n* `RETURN p.id, COUNT(p)`: This line returns the IDs of the papers and the count of papers in the database. The `COUNT(p)` function returns the number of papers that',
 "  Sure! Here's a Cypher query to count the number of authors in the graph:\n```\nMATCH (a:Author) RETURN COUNT(a)\n```\nExplanation:\n\n* `MATCH` is the Cypher command that matches nodes in the graph that match the specified pattern. In this case, we're matching nodes labeled with `Author`.\n* `RETURN` is the command that returns the resu

In [ ]:
df_test_all['llama_generations'] = generations_raw

In [ ]:
df_test_all.to_csv('modified1.csv')

In [ ]:
df_test_all['llama_generations']

0        To answer the question "How many papers are ...
1        Sure! Here's a Cypher query to count the num...
2        Sure! To answer the question "How many confe...
3        To find all papers authored by "Daphne Kolle...
4        To answer the question "How many papers cite...
                             ...                        
214      To find the papers on Maximum Likelihood tha...
215      To identify recent popular papers in Augment...
216      To find the trending research topics in Supe...
217      Sure! Here's a Cypher query to find the late...
218      To answer the question "Who are the most act...
Name: llama_generations, Length: 219, dtype: object

In [ ]:
import re
pattern = r'```.*?```'
df_test_all['llama_cleaned_generations'] = df_test_all['llama_generations'].apply(
    lambda x:re.findall(pattern, x, re.DOTALL) )
df_test_all['llama_cleaned_generations'] = df_test_all['llama_cleaned_generations'].apply(
    lambda x: x[0].replace("```", '') if len(x)>0 else x)
df_test_all['llama_cleaned_generations']

0              \nMATCH (p:Paper) RETURN p.id, COUNT(p)\n
1                   \nMATCH (a:Author) RETURN COUNT(a)\n
2                 \nMATCH (c:Conference) RETURN c.name\n
3      \nMATCH (a:Author {name: "Daphne Koller"})-[:A...
4      \nMATCH (p1:Paper)-[:CITES {num_cites: > 1}]- ...
                             ...                        
214    \nMATCH (p:Paper)-[:PRESENTED_AT]->(conf:Confe...
215    \nMATCH (paper:AugmentedReality)-[:PUBLISHED]-...
216                                                   []
217    \nMATCH (p:Paper)-[:PUBLISHED]->()\nWHERE p.to...
218    \nMATCH (n:Author)-[:PUBLISHED {conference: "c...
Name: llama_cleaned_generations, Length: 219, dtype: object

In [ ]:
# df_test_all.to_csv('content/drive/MyDrive/cypher/modified1.csv')
df_test_all = pd.read_csv('llm_all_v1.csv')

In [ ]:
df_test_all['llama_generations'] = generations_raw

In [ ]:
pattern = r'```.*?```'
df_test_all['llama_cleaned_generations'] = df_test_all['llama_generations'].apply(
    lambda x:re.findall(pattern, x, re.DOTALL) )
df_test_all['llama_cleaned_generations'] = df_test_all['llama_cleaned_generations'].apply(
    lambda x: x[0].replace("```", '') if len(x)>0 else x)
df_test_all['llama_cleaned_generations']


In [ ]:
df_test_all.to_csv('llm_all_v2.csv')